In [1]:
#导入jqdata的全部函数
from jqdata import *
#使用get_money_flow函数获取
df = get_money_flow('002458.XSHE',
                    fields = ['date',
                             #股票代码
                           'sec_code',
                             #涨跌幅
                           'change_pct',
                             #主力金额，包括超大单和大单
                           'net_amount_main',
                             #主力成交额占总成交额的比例
                           'net_pct_main'],
                    #设置好起止日期
                    start_date = '2018-04-09',
                    end_date = '2020-04-08')
#检查是否成功
df.head()

,date,sec_code,change_pct,net_amount_main,net_pct_main
0,2018-04-09,002458.XSHE,-0.84,-570.2092,-8.7478
1,2018-04-10,002458.XSHE,-2.55,-360.5189,-4.4665
2,2018-04-11,002458.XSHE,0.44,-601.2525,-6.2953
3,2018-04-12,002458.XSHE,-1.01,168.3327,2.9341
4,2018-04-13,002458.XSHE,-0.73,-81.8302,-1.5589


In [2]:
#增加一个字段，记录股价上涨还是下跌
#如果股价上涨，则以1标记，否则以0标记
df['up_or_down'] = np.where(df['change_pct']>0,1,0)
#在增加一个字段，记录主力资金净流入还是流出
#如果净流入，标记为1，否则标记为0
df['money_in_out'] = np.where(df['net_amount_main']>0,1,0)
#检查是否成功
df.head()

,date,sec_code,change_pct,net_amount_main,net_pct_main,up_or_down,money_in_out
0,2018-04-09,002458.XSHE,-0.84,-570.2092,-8.7478,0,0
1,2018-04-10,002458.XSHE,-2.55,-360.5189,-4.4665,0,0
2,2018-04-11,002458.XSHE,0.44,-601.2525,-6.2953,1,0
3,2018-04-12,002458.XSHE,-1.01,168.3327,2.9341,0,1
4,2018-04-13,002458.XSHE,-0.73,-81.8302,-1.5589,0,0


In [3]:
#瓦氏因子来了，用两个自增的字段相乘，得出因子值
df['factor_wa'] = df['up_or_down'] * df['money_in_out']
#再把次日涨跌作为预测标签存储到‘next_day’字段
df['next_day'] = df['up_or_down'].shift(-1)
#检查是否成功
df.head()

,date,sec_code,change_pct,net_amount_main,net_pct_main,up_or_down,money_in_out,factor_wa,next_day
0,2018-04-09,002458.XSHE,-0.84,-570.2092,-8.7478,0,0,0,0.0
1,2018-04-10,002458.XSHE,-2.55,-360.5189,-4.4665,0,0,0,1.0
2,2018-04-11,002458.XSHE,0.44,-601.2525,-6.2953,1,0,0,0.0
3,2018-04-12,002458.XSHE,-1.01,168.3327,2.9341,0,1,0,0.0
4,2018-04-13,002458.XSHE,-0.73,-81.8302,-1.5589,0,0,0,0.0


In [4]:
#还是请出已经熟悉的KNN算法
from sklearn.neighbors import KNeighborsClassifier
#导入数据集拆分工具
from sklearn.model_selection import train_test_split
#当然还需要有pandas
import pandas as pd
#数据集中把日期、股票代码，以及我们添加的特征去掉
dataset = df.drop(['date', 
                   'sec_code',
                   'up_or_down',
                   'money_in_out'],
                  axis=1)
#检查是否成功
dataset.head()

,change_pct,net_amount_main,net_pct_main,factor_wa,next_day
0,-0.84,-570.2092,-8.7478,0,0.0
1,-2.55,-360.5189,-4.4665,0,1.0
2,0.44,-601.2525,-6.2953,0,0.0
3,-1.01,168.3327,2.9341,0,0.0
4,-0.73,-81.8302,-1.5589,0,0.0


In [5]:
#将‘next_day’以外的字段，作为数据集的特征
X = dataset.drop(['next_day'],axis=1)[:-1]
#将‘next_day’作为数据集的标签
y = dataset['next_day'][:-1]
#将数据集拆分为训练集与验证集
X_train, X_test, y_train, y_test =\
train_test_split(X, y, random_state = 28)

In [6]:
#创建KNN分类器，n_neighbors参数依然取95
knn = KNeighborsClassifier(n_neighbors=95)
#使用训练集训练模型
knn.fit(X_train, y_train)
#打印训练集中模型准确率
print(knn.score(X_train, y_train))
#打印验证集中模型准确率
print(knn.score(X_test,y_test))

0.5671232876712329
0.5573770491803278


5.2

In [1]:
#这里需要导入聚宽因子库的get_factor_values函数
from jqfactor import get_factor_values
#当然后面也要用到pandas
import pandas as pd

思路是选超大盘股，净利润率高，且高速成长的

In [3]:
#指定get_all_securities的types参数为index
#即可查询全部指数
indices = get_all_securities(types=['index'])
#查看前十条结果
indices.head(20)

,display_name,name,start_date,end_date,type
000001.XSHG,上证指数,SZZS,1991-07-15,2200-01-01,index
000002.XSHG,A股指数,AGZS,1992-02-21,2200-01-01,index
000003.XSHG,B股指数,BGZS,1992-02-21,2200-01-01,index
000004.XSHG,工业指数,GYZS,1993-05-03,2200-01-01,index
000005.XSHG,商业指数,SYZS,1993-05-03,2200-01-01,index
000006.XSHG,地产指数,DCZS,1993-05-03,2200-01-01,index
000007.XSHG,公用指数,GYZS,1993-05-03,2200-01-01,index
000008.XSHG,综合指数,ZHZS,1993-05-03,2200-01-01,index
000009.XSHG,上证380,SZ380,2010-11-29,2200-01-01,index
000010.XSHG,上证180,SZ180,2002-07-01,2200-01-01,index


In [15]:
#导入聚宽的因子分析库
from jqfactor import analyze_factor
#使用get_factor_values函数获取上证50成分股的市值
factor_mc=get_factor_values(securities=get_index_stocks('000016.XSHG'), factors=['market_cap'],
                  end_date='2020-04-30',count=1)['market_cap']
#检查结果
factor_mc.T.head()

,2020-04-30 00:00:00
code,
600000.XSHG,3.120126e+11
600009.XSHG,1.362360e+11
600016.XSHG,2.578784e+11
600028.XSHG,5.399776e+11
600030.XSHG,3.063646e+11


In [16]:
#get_factor_values中
#factors参数传入cash_flow_to_price_ratio
#即可获得市现率的倒数
factor_cfp = get_factor_values(securities = get_index_stocks('000016.XSHG'),
                           factors = ['cash_flow_to_price_ratio'],
                              end_date = '2020-04-30',
                              count = 1)['cash_flow_to_price_ratio']
#检查结果
factor_cfp.T.head(5)

,2020-04-30 00:00:00
code,
600000.XSHG,0.060232
600009.XSHG,0.006489
600016.XSHG,-0.088220
600028.XSHG,-0.046224
600030.XSHG,0.270966


In [17]:
#在get_factor_values函数中
#使用net_profit_ratio作为factors参数
#即可查询到企业的净利润率
factor_npr = get_factor_values(securities = get_index_stocks('000016.XSHG'),
                              factors = ['net_profit_ratio'],
                              end_date = '2020-04-30',
                              count = 1)['net_profit_ratio']
#检查结果
factor_npr.T.head()

,2020-04-30 00:00:00
code,
600000.XSHG,0.308175
600009.XSHG,0.401260
600016.XSHG,0.299960
600028.XSHG,0.011633
600030.XSHG,0.275232


In [18]:
#在get_factor_values函数中
#使用net_profit_growth_rate作为factors参数
#即可查询到企业的净利润增长率
factor_npgr = get_factor_values(securities = get_index_stocks('000016.XSHG'),
                               factors = ['net_profit_growth_rate'],
                               end_date = '2020-04-30',
                               count = 1)['net_profit_growth_rate']
#检查结果
factor_npgr.T.head()

,2020-04-30 00:00:00
code,
600000.XSHG,0.029481
600009.XSHG,-0.182414
600016.XSHG,0.089774
600028.XSHG,-0.560616
600030.XSHG,0.094400


In [19]:
#新建一个DataFrame，和前面市值数据保持同样的序号
factors = pd.DataFrame(index = factor_mc.T.index)
#在新的DataFrame中创建4个字段
#分别把市值、市现率倒数、净利润率、净利润增长率存储到其中
factors['mc'] = factor_mc.T['2020-04-30 00:00:00']
factors['cfp'] = factor_cfp.T['2020-04-30 00:00:00']
factors['npr'] = factor_npr.T['2020-04-30 00:00:00']
factors['npgr'] = factor_npgr.T['2020-04-30 00:00:00']
#检查结果
factors.head()

,mc,cfp,npr,npgr
code,,,,
600000.XSHG,3.120126e+11,0.060232,0.308175,0.029481
600009.XSHG,1.362360e+11,0.006489,0.401260,-0.182414
600016.XSHG,2.578784e+11,-0.088220,0.299960,0.089774
600028.XSHG,5.399776e+11,-0.046224,0.011633,-0.560616
600030.XSHG,3.063646e+11,0.270966,0.275232,0.094400


In [20]:
#为了计算，先把数据中的空值去掉
factors = factors.dropna()
#检查下是否还有空值
factors.isnull().sum()

mc      0
cfp     0
npr     0
npgr    0
dtype: int64

In [21]:
#因为各因子数值的量纲差异较大
#需要做一点简单的缩放处理
from sklearn.preprocessing import StandardScaler
#导入scikit-learn中的PCA主成分分析工具
from sklearn.decomposition import PCA
#创建StandardScaler实例，会将数据量纲压缩到同一个区间中
scaler = StandardScaler()
#使用StandardScaler缩放原始的因子值
factors_scl = scaler.fit_transform(factors.drop('mc',axis=1))
#接下来使用PCA，提取主成分数量指定为1
pca = PCA(n_components = 1)
#使用缩放后的数据进行拟合
pca.fit(factors_scl)
#查看pca给各因子分配的权重
pca.components_

array([[0.6501248793028565, 0.6737126962673577, 0.35135287703335166]])

In [22]:
#在factors数据表中添加一个pca字段
#存储提取出来的主成分
factors['pca'] = pca.transform(factors_scl)
#看一下主成分数值最高的5只股票
factors.sort_values(by='pca', ascending = False).head(10)

,mc,cfp,npr,npgr,pca
code,,,,,
601166.XSHG,3.452670e+11,0.800000,0.370036,0.075121,3.770163
601288.XSHG,1.210941e+12,0.448029,0.335705,0.045967,2.182471
601398.XSHG,1.842620e+12,0.402414,0.372866,0.044122,2.179109
601939.XSHG,1.607571e+12,0.325722,0.374030,0.049342,1.883905
601988.XSHG,1.024470e+12,0.323248,0.365705,0.042991,1.828934
601688.XSHG,1.646504e+11,0.186306,0.359591,0.515651,1.622891
601066.XSHG,2.675470e+11,0.069972,0.400202,0.661239,1.469796
600519.XSHG,1.589970e+12,0.014857,0.520656,0.130552,1.419197
600036.XSHG,8.849644e+11,0.219173,0.345430,0.146780,1.398080


In [23]:
factors.sort_values(by='pca', ascending = False).head(50)

,mc,cfp,npr,npgr,pca
code,,,,,
601166.XSHG,3.452670e+11,0.800000,0.370036,0.075121,3.770163
601288.XSHG,1.210941e+12,0.448029,0.335705,0.045967,2.182471
601398.XSHG,1.842620e+12,0.402414,0.372866,0.044122,2.179109
601939.XSHG,1.607571e+12,0.325722,0.374030,0.049342,1.883905
601988.XSHG,1.024470e+12,0.323248,0.365705,0.042991,1.828934
601688.XSHG,1.646504e+11,0.186306,0.359591,0.515651,1.622891
601066.XSHG,2.675470e+11,0.069972,0.400202,0.661239,1.469796
600519.XSHG,1.589970e+12,0.014857,0.520656,0.130552,1.419197
600036.XSHG,8.849644e+11,0.219173,0.345430,0.146780,1.398080
